In [1]:
import xarray as xr
import reverse_geocoder as rg
from tqdm import tqdm
import pandas as pd
import itertools
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
import pickle as pkl

In [13]:
country_size={"DE":357600,"AT":83879,"SE":450295,"FR":543941,"IT":301958,"PL":312679,"ES":498485,"NL":41543,"BE":30528}

In [14]:
data=xr.open_dataset(filename_or_obj=r"C:\Daten\Foschung\RiskAware\Data\Weather\rainfall\gpcp_v01r03_daily_d20221208.nc",engine="netcdf4")
geocoding=pkl.load(open(r"country_dict.pkl","rb"))
latitudes = data.latitude.values
longitudes = data.longitude.values

In [15]:
dates=pd.date_range(start="2022-01-01",end="2022-12-31",freq="D")

In [25]:
for date in dates:
    date_str = date.strftime('%Y%m%d')
    data = xr.open_dataset(f"C:\\Daten\\Foschung\\RiskAware\\Data\\Weather\\rainfall\\gpcp_v01r03_daily_d{date_str}.nc", engine='netcdf4')
    precip = pd.DataFrame(np.array(data.precip).reshape(37, 39), index=latitudes, columns=longitudes)
    precip[precip < 0] = 0
    country_precip = pd.DataFrame(index=dates, columns=country_size.keys())
    
    for country in country_size.keys():
        country_keys = [k for k, v in geocoding.items() if v == country]
        total_precip = 0
        
        for key in country_keys:
            if key[1] < 0:
                key = (key[0], key[1] + 360)
            total_precip += precip.loc[key] * np.cos(key[0]) * pow(111320, 2) * (country_size[country] / (len(country_keys) * np.cos(key[0]) * pow(111.32, 2)))
        
        country_precip.loc[date, country] = total_precip

In [28]:
country_precip.loc[date,country]

np.float32(118990090000.0)

In [63]:
# save the country dictionary
# as a pickle
import pickle
with open('country_dict.pkl', 'wb') as f:
    pickle.dump(country, f)